In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Initialisation du géolocalisateur avec un délai plus long et un limiteur de taux
geolocator = Nominatim(user_agent="geoapi", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3)

# Fonction pour obtenir les coordonnées
def get_coordinates(country_name):
    try:
        location = geocode(country_name)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur lors de la récupération des coordonnées pour {country_name}: {e}")
        return None, None

# Chargement du fichier CSV original
input_csv = "/content/vih_no_of_cases_adults_15_to_49_by_country_clean.csv"
df_original = pd.read_csv(input_csv)

# Extraction des noms uniques de pays (insensibles à la casse)
unique_countries = df_original.iloc[:, 0].str.lower().drop_duplicates().str.replace(r"\(.*?\)", "", regex=True).str.strip()

# Création d'un DataFrame pour les noms uniques de pays
unique_countries_df = pd.DataFrame({'Country': unique_countries})

# Ajout des colonnes Latitude et Longitude
unique_countries_df[['Latitude', 'Longitude']] = unique_countries_df['Country'].apply(
    lambda country: pd.Series(get_coordinates(country))
)

# Chargement des coordonnées dans un fichier temporaire (facultatif)
output_csv = "unique_country_coordinates.csv"
unique_countries_df.to_csv(output_csv, index=False)

# Préparation des données pour la fusion (normalisation des noms de pays)
df_original['Country_cleaned'] = df_original.iloc[:, 0].str.lower().str.replace(r"\(.*?\)", "", regex=True).str.strip()
unique_countries_df.rename(columns={'Country': 'Country_cleaned'}, inplace=True)

# Fusion des données originales avec les coordonnées
df_merged = df_original.merge(unique_countries_df, on='Country_cleaned', how='left')

# Suppression de la colonne temporaire
df_merged.drop(columns=['Country_cleaned'], inplace=True)

# Sauvegarde du fichier avec les coordonnées ajoutées
output_with_coordinates = "original_with_coordinates.csv"
df_merged.to_csv(output_with_coordinates, index=False)

print(f"Les coordonnées ont été ajoutées et enregistrées dans {output_with_coordinates}")


Les coordonnées ont été ajoutées et enregistrées dans original_with_coordinates.csv
